In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import src.config as config

import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/423065/fs/418888/fv/time_series_hourly_feature_view/version/1


In [6]:
ts_data, _ = feature_view.training_data(
    description='Times-series hourly taxi rides',
)

ts_data.sort_values(by=["pickup_location_id","pickup_hour"], inplace=True)
ts_data

Finished: Reading data from Hopsworks, using ArrowFlight (9.52s) 


,pickup_hour,rides,pickup_location_id
1809206,2023-01-01 00:00:00+00:00,0,1
828891,2023-01-01 01:00:00+00:00,0,1
1980301,2023-01-01 02:00:00+00:00,0,1
783947,2023-01-01 03:00:00+00:00,0,1
1856621,2023-01-01 04:00:00+00:00,0,1
...,...,...,...
986409,2024-01-27 01:00:00+00:00,3,265
981932,2024-01-27 02:00:00+00:00,5,265
986639,2024-01-27 03:00:00+00:00,5,265
985843,2024-01-27 04:00:00+00:00,8,265


In [7]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28,
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 262/262 [00:31<00:00,  8.19it/s]


f{features_and_target.shape=}


In [8]:
print(f'{features_and_target.shape=}')

features_and_target.shape=(91550, 675)


In [29]:
from datetime import timedelta, datetime
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months

cutoff_date = timezone('UTC').localize((datetime.today() - timedelta(days=28*1)))
# cutoff_date = pd.to_datetime(datetime.now() - timedelta(days=28*1))

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=datetime.datetime(2023, 12, 29, 23, 21, 2, 681224, tzinfo=<UTC>)
X_train.shape=(84097, 674)
y_train.shape=(84097,)
X_test.shape=(7453, 674)
y_test.shape=(7453,)
